# Map of Gordon's Bay - Refactored Version

## For Prodive's Divemaster course

As part of this course, I need to draw a map of a dive site. Being an odd fish, I've got a bit carried away. There's a lot more context in the [readme file](https://github.com/notionparallax/dive-map/blob/main/README.md) in the repo.

Let's get started. If you're just following along for the pictures, ignore all the python (the coloured writing) and go straight to the pictures.

**This is the refactored version** that uses the new modular architecture for better maintainability and reusability.

In [ ]:
# pip install -r requirements.txt

In [ ]:
# Import standard libraries
import math
import os
from functools import partial

import contextily as cx
import folium
import geopandas as gp
import matplotlib.colors
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from folium import plugins
from geopy import Point as geopy_pt
from geopy.distance import geodesic
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely import centroid
from shapely.geometry import LineString, MultiPoint, Point, Polygon

import config

# Import our refactored modules
from config import (
    BOTTOM_CONDITION_COLORS,
    CONTOUR_SPACING,
    CRS,
    DEPTH_RANGE,
    FIGURE_SIZE,
    GORDONS_BAY_COORDS,
    SHORE_DEPTHS,
    TEXT_COLOUR,
    SPEAR_FISHING_BOUNDARY_COORDS,
)
from data_loaders import DiveDataProcessor

# Import existing photo metadata (until we refactor this too)
from photo_meta import photo_meta as pm_1
from photo_meta_day_2 import photo_meta as pm_2
from photo_meta_day_3 import photo_meta as pm_3
from refactored_functions import (
    add_note_label,
    apply_annotations,
    draw_shortcut_arrow,
    filter_by_distance,
    make_marker_text,
    measure_line_string,
    naieve_ffill,
)
from visualization import ContourRenderer, MapRenderer, MarkerRenderer, ScaleBarRenderer

# Set matplotlib parameters
plt.rcParams["svg.fonttype"] = "none"

print("✅ All modules imported successfully!")

In [ ]:
# Consolidate photo metadata 
# I've got the photo data as python data, rather than json, not really for any good reason though. 
# It's just a big list of dictionaries. These are made by running img.py on a folder full of photos
photo_meta = pm_1 + pm_2 + pm_3

# Configure data sources - now centralized instead of scattered throughout
fit_files = [
    os.path.join("fit", "ScubaDiving_2024-03-08T09_29_45.fit"),
    os.path.join("fit", "ScubaDiving_2024-03-08T11_26_21.fit"),
    os.path.join("fit", "ScubaDiving_2024-03-23T08_51_29.fit"),
    os.path.join("fit", "ScubaDiving_2024-03-23T09_41_59.fit"),
    os.path.join("fit", "ScubaDiving_2024-03-23T11_06_51.fit"),
    os.path.join("fit", "ScubaDiving_2024-05-17T10_33_57.fit"),
    os.path.join("fit", "ScubaDiving_2024-05-24T09_04_22.fit"),
]

# GPX configurations with their specific parameters
gpx_configs = [
    {
        "file_path": os.path.join("gps", "20240308-090746 - Gordons.gpx"),
        "description": "chain_loop",
        "crop": True,
        "end_time": "2024-03-08T02:25:26Z",
        "dive_end_time_delta": 180,
        "dive_start_time_delta": 250,
    },
    {
        "file_path": os.path.join("gps", "20240308-090746 - Gordons.gpx"),
        "description": "boulder_garden",
        "crop": True,
        "end_time": "2024-03-08T02:25:26Z",
        "dive_end_time_delta": 70,
        "dive_start_time_delta": 120,
    },
    {
        "file_path": os.path.join("gps", "20240323-081550 - Map dive Saturday morning.gpx"),
        "description": "wall_to_desert",
        "crop": True,
        "end_time": "2024-03-22 22:52:06+00:00",
        "dive_end_time_delta": 5,
        "dive_start_time_delta": 65,
    },
    {
        "file_path": os.path.join("gps", "20240323-104518 - Dive 2.gpx"),
        "description": "far_side_desert",
        "crop": True,
        "end_time": "2024-03-23 01:14:03.999000+00:00",
        "dive_end_time_delta": 10,
        "dive_start_time_delta": 68,
    },
    {
        "file_path": os.path.join("gps", "20240517-101408 - Map random swim.gpx"),
        "description": "random_swim",
        "crop": False,
    },
    {
        "file_path": os.path.join("gps", "20240524-084346 - Bommie.gpx"),
        "description": "out to the bommie",
        "crop": False,
    },
]

print(f"📊 Configured {len(fit_files)} FIT files and {len(gpx_configs)} GPS tracks")

## Data Loading with New Architecture

The depth data comes from my watch, a Suunto D5, and I export the `.fit` file from the phone app.

Instead of manually loading and combining each data source separately, we now use the `DiveDataProcessor` class to handle all data loading consistently.

In [ ]:
# Load all data with the new streamlined approach
processor = DiveDataProcessor()
depth_df, dives_gdf, photo_df = processor.load_all_data(
    fit_files=fit_files,
    gpx_configs=gpx_configs,
    photo_metadata=photo_meta
)

print(f"📈 Loaded data:")
print(f"   • {len(depth_df):,} depth measurements")
print(f"   • {len(dives_gdf):,} GPS positions across {len(dives_gdf.description.unique())} dive tracks")
print(f"   • {len(photo_df):,} photos with metadata")
print(f"   • Date range: {dives_gdf.index.min()} to {dives_gdf.index.max()}")

In [ ]:
# Show individual depth profiles (preserving the original analysis approach)
# Create subplots for each set of dives as in the original

# Depth profile for day 1: chain and boulder garden dives
depth_day_1 = depth_df[depth_df.source_file.str.contains("2024-03-08")]
if not depth_day_1.empty:
    depth_day_1.plot(
        y='depth', 
        figsize=(12, 6),
        title="Depth of the dives - Day 1\n1 around the gordon's chain, 2 around the boulder garden",
        ylabel="Depth (m)",
        xlabel="Time (UTC)"
    )
    plt.tight_layout()
    plt.show()

# Depth profile for day 2: wall to desert dives  
depth_day_2 = depth_df[depth_df.source_file.str.contains("2024-03-23")]
if not depth_day_2.empty:
    depth_day_2.plot(
        y='depth',
        figsize=(12, 6), 
        title="Depth of the dives - Day 2\n1 around bottom of the wall/desert interface,\n2 across to the other side",
        ylabel="Depth (m)",
        xlabel="Time (UTC)"
    )
    plt.tight_layout()
    plt.show()

# Day 3: Random swim (with surface trips filtered out)
depth_day_3 = depth_df[depth_df.source_file.str.contains("2024-05-17")]
if not depth_day_3.empty:
    # I got a bit disorientated and seasick, and I got my fin strap entangled on the
    # float line, so there's a couple of trips to the surface. So that they don't
    # get treated like high spots, I'm going to crop off any readings shallower than 2m.
    filtered_depth_3 = depth_day_3[depth_day_3.depth < -2]
    filtered_depth_3.plot(
        y='depth',
        figsize=(12, 6),
        title="Depth of the dive - Day 3\n(Surface trips filtered out)",
        ylabel="Depth (m)", 
        xlabel="Time (UTC)"
    )
    plt.tight_layout()
    plt.show()

# Day 4: Bommie dive
depth_day_4 = depth_df[depth_df.source_file.str.contains("2024-05-24")]
if not depth_day_4.empty:
    depth_day_4.plot(
        y='depth',
        figsize=(12, 6),
        title="Depth of the dive - Day 4",
        ylabel="Depth (m)",
        xlabel="Time (UTC)"
    )
    plt.tight_layout()
    plt.show()

print("📊 Individual dive depth profiles displayed")

## GPS Track Analysis

The GPS data comes from an app on my phone. This odd shape is because I forgot to turn it off and then we drove back to the shop. If we look at the numbers on the x axis, we've almost gone 0.1 of a degree, which in metric is:

```
Decimal Places   Aprox. Distance    Say What?
1                10 kilometers      6.2 miles
2                1 kilometer        0.62 miles
3                100 meters         About 328 feet
4                10 meters          About 33 feet
5                1 meter            About 3 feet
6                10 centimeters     About 4 inches
7                1.0 centimeter     About 1/2 an inch
8                1.0 millimeter     The width of paperclip wire.
9                0.1 millimeter     The width of a strand of hair.
10               10 microns         A speck of pollen.
11               1.0 micron         A piece of cigarette smoke.
12               0.1 micron         You're doing virus-level mapping at this point.
13               10 nanometers      Does it matter how big this is?
14               1.0 nanometer      Your fingernail grows about this far in one second.
15               0.1 nanometer      An atom. An atom! What are you mapping?
```

from [here](https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude)

But we don't care about driving around the Eastern Suburbs, so we'll have to crop it off. Also, this is both dives, so we'll need to split those out too.

In [ ]:
# Visualize all dive tracks (now handled cleanly by our data loader)
fig, ax = plt.subplots(figsize=(15, 10))
dives_gdf.plot(column="description", ax=ax, legend=True, alpha=0.7, markersize=2, marker="x")
ax.set_title(f"All dive tracks ({', '.join(dives_gdf.description.unique())})")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.tight_layout()
plt.show()


In [ ]:
# Show individual dive tracks as in original
for desc in dives_gdf.description.unique():
    track_data = dives_gdf[dives_gdf.description == desc]
    if not track_data.empty:
        fig, ax = plt.subplots(figsize=(10, 8))
        track_data.plot(ax=ax, alpha=0.6, markersize=1)
        ax.set_title(f"Dive Track: {desc}")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
        plt.tight_layout()
        plt.show()
        print(f"📍 {desc}: {len(track_data)} GPS points")

## Photo Data Integration

The glue that joins it all up is photos - the photos tell us when we were at a feature. And in this case, what the bottom condition was like.

In [ ]:
# Timezone unification and data integration
# There are some tricky bits because everything is in different time zones, so let's unify them all into UTC
print("Before timezone conversion:")
print(f"\tdives_df: {repr(dives_gdf.iloc[0].name)}")
print(f"\tdepth_df: {repr(depth_df.iloc[0].name)}")  
print(f"\tphoto_df: {repr(photo_df.iloc[0].name)}")

# Convert all timestamps to UTC (the data loader should handle this, but ensuring consistency)
dives_gdf.index = dives_gdf.index.tz_convert("UTC")
depth_df.index = depth_df.index.tz_convert("UTC")  
photo_df.index = photo_df.index.tz_convert("UTC")

print("\nAfter timezone conversion:")
print(f"\tdives_df: {repr(dives_gdf.iloc[0].name)}")
print(f"\tdepth_df: {repr(depth_df.iloc[0].name)}")
print(f"\tphoto_df: {repr(photo_df.iloc[0].name)}")

# Add source identifiers
dives_gdf["source"] = "dives" 
depth_df["source"] = "depth"
photo_df["source"] = "photo"

## Data Consolidation

If for some reason the data is too heavy, use this to chop it down. It's not doing any chopping at the moment.

The DataFrame below is from mixing all the data together. There are a lot more GPS signals than anything else, and a lot more depth values than photos. They're all time sorted, and then the preceding value is filled down until there's another one to take over.

In [ ]:
# Consolidate all data sources
reduced_dives = dives_gdf
# reduced_dives = reduced_dives.iloc[::60]  # pick one frame a minute
# reduced_dives = reduced_dives[
#     reduced_dives.index > depth_df.index[0]
# ]  # wait until there's depth data

all_df = pd.concat([reduced_dives, depth_df, photo_df])
all_df.sort_index(axis=0, inplace=True)

# Forward fill to propagate values between sparse measurements
all_df["depth"] = all_df["depth"].ffill()
all_df["depth"] = all_df["depth"].fillna(0)
all_df["filename"] = all_df["filename"].ffill(limit=10)

# Use our refactored naive_ffill function for geometry
naieve_ffill(all_df, "geometry")

all_df["description"] = all_df["description"].ffill()
all_df.drop(["lat", "lon"], axis=1, inplace=True, errors="ignore")

# Convert to GeoDataFrame
all_gdf = gp.GeoDataFrame(all_df)

print(f"📊 Consolidated dataset: {len(all_gdf):,} total records")
print(f"   • Sources: {', '.join(all_gdf.source.value_counts().to_string().split())}")
all_gdf.head()

In [ ]:
# Process markers (numbered buoys and intermediate points)


markers_df = all_gdf[
    (all_gdf.source == "photo")
    & ((all_gdf.marker_type == "numbered") | (all_gdf.marker_number != ""))
].copy()

intermediate_df = all_gdf[
    (all_gdf.source == "photo") & (all_gdf.marker_type == "intermediate")
].copy()

# Use our refactored marker text function
markers_df.loc[:, "marker_text"] = markers_df.apply(make_marker_text, axis=1)

# Create unified marker positions by grouping photos of the same marker
uni_marker_df = (
    markers_df.groupby("marker_number")
    .apply(
        lambda grp: pd.Series(
            {
                "geometry": centroid(MultiPoint(list(grp.geometry))),
                "marker_text": grp.marker_text.iloc[0],
                "depth": grp.depth.mean(),
            }
        )
    )
    .sort_index()
)

print(f"🎯 Processed markers:")
print(f"   • {len(uni_marker_df)} numbered markers")
print(f"   • {len(intermediate_df)} intermediate markers")
uni_marker_df

In [ ]:
# REFACTORED MAP GENERATION (Using Modular Architecture)
# Clean approach using the refactored visualization classes and functions

print("🗺️  Starting refactored map generation...")

# Create map renderer and initialize figure
map_renderer = MapRenderer(FIGURE_SIZE)
fig, ax = map_renderer.create_figure()

# Add scalebar using the modular approach
starting_point = geopy_pt(-33.9175, 151.265)
scalebar_distances = [0, 5, 10, 15, 20, 50, 100]
map_renderer.add_scalebar(starting_point, scalebar_distances)

# Add north arrow using the modular approach
n_bottom_pt = geopy_pt(-33.9175, 151.267)
map_renderer.add_north_arrow(n_bottom_pt)

# Add colorbar for depth
divider = make_axes_locatable(ax)
cax_cb = divider.append_axes("right", size="2%", pad=0.05)
sm = plt.cm.ScalarMappable(cmap="rainbow", norm=plt.Normalize(vmin=-14, vmax=0))
cbar = plt.colorbar(sm, cax=cax_cb, label="Depth (m)")

# Process bottom conditions using refactored functions
bottom_gdf = all_gdf[
    (all_gdf.source == "photo") & (all_gdf.bottom_condition != "unspecified")
]

# Use the refactored filter_by_distance function
filtered_gdf = filter_by_distance(bottom_gdf, min_distance=1.5)

# Load and add click conditions (shore data) - this is what was missing!
try:
    if os.path.exists("click_conditions.json"):
        json_df = pd.read_json("click_conditions.json")
        click_gdf = gp.GeoDataFrame(
            geometry=json_df.apply(lambda row: Point(row.lon, row.lat), axis=1)
        )
        click_gdf["bottom_condition"] = json_df.condition
        click_gdf.set_crs(all_gdf.crs, inplace=True)

        # Concatenate with filtered photo data
        filtered_gdf = pd.concat([filtered_gdf, click_gdf], ignore_index=True)
        print("📍 Added click conditions (shore data)")
except Exception as e:
    print(f"⚠️  Click conditions file not found: {e}")

# Apply colors using the config
filtered_gdf["colour"] = filtered_gdf["bottom_condition"].apply(
    lambda x: BOTTOM_CONDITION_COLORS.get(x, "deeppink")
)

# Plot bottom condition markers
filtered_gdf.plot(color=filtered_gdf.colour, ax=ax, markersize=1, alpha=0.8)
print(f"🎨 Added {len(filtered_gdf)} bottom condition markers")

# Add contours using the modular renderer WITH SHORE CONDITIONS INTEGRATION
print("🔧 Preparing contour data with shore conditions...")

# Step 1: Get underwater depth data
underwater_contour_gdf = all_gdf[
    all_gdf.depth.notnull() & (all_gdf.depth < -1.5)
].copy()

# Step 2: Prepare shore conditions with depths (exactly like original map.ipynb)
shore_conditions = list(SHORE_DEPTHS.keys())
shore_gdf = click_gdf[click_gdf.bottom_condition.isin(shore_conditions)].copy()
shore_gdf["depth"] = shore_gdf["bottom_condition"].map(SHORE_DEPTHS, na_action="ignore")

# Step 3: Concatenate shore and underwater data (the critical missing step!)
contour_gdf = pd.concat([underwater_contour_gdf, shore_gdf], ignore_index=True)

print(
    f"✅ Contour data prepared: {len(contour_gdf)} records (depth range: {contour_gdf.depth.min():.1f}m to {contour_gdf.depth.max():.1f}m)"
)
print(f"   • Underwater records: {len(underwater_contour_gdf)}")
print(f"   • Shore condition records: {len(shore_gdf)}")

if len(contour_gdf) > 0:
    try:
        contour_renderer = ContourRenderer()
        levels = list(range(-15, 0, 1))
        contour_renderer.plot_contours(ax, contour_gdf, levels)
        print("📈 Generated depth contours WITH shore conditions")
    except Exception as e:
        print(f"⚠️  Could not generate contours: {e}")

# Add satellite basemap using the map renderer
try:
    # Try Google Satellite first (highest quality)
    google_source = cx.providers.GoogleTiles(api_key=None, variant="satellite")
    cx.add_basemap(ax, crs=all_gdf.crs, source=google_source)
    print("🛰️  Added Google satellite basemap (high resolution)")
except Exception as e:
    print(f"Couldn't add high res Google basemap because: {e}")
    try:
        # Fallback to custom Google tile URL
        google_tiles = {
            "url": "https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
            "attribution": "Google Satellite",
            "name": "Google Satellite",
        }
        cx.add_basemap(
            ax,
            crs=all_gdf.crs,
            source=google_tiles["url"],
            attribution=google_tiles["attribution"],
        )
        print("🛰️  Added Google satellite basemap (custom URL)")
    except Exception as e:
        try:
            # Final fallback to Esri
            cx.add_basemap(ax, crs=all_gdf.crs, source=cx.providers.Esri.WorldImagery)
            print("🛰️  Added Esri satellite basemap (fallback)")
        except Exception as e2:
            print(f"⚠️  Could not add any basemap: {e2}")

# Add numbered markers using the modular marker renderer
marker_renderer = MarkerRenderer()
for _, row in uni_marker_df.iterrows():
    marker_renderer.add_numbered_marker(ax, row)
    marker_renderer.add_tolerance_circle(ax, row)

print(f"🎯 Added {len(uni_marker_df)} numbered markers with tolerance circles")

# Add notes if they exist
notes_gdf = all_gdf[all_gdf.note.notnull()]
if not notes_gdf.empty:
    for _, row in notes_gdf.iterrows():
        apply_annotations(pd.DataFrame([row]), ax, add_note_label)
    print(f"📝 Added {len(notes_gdf)} note annotations")

# Plot the chain trail line
chain_pts = uni_marker_df.geometry.copy()
chain_pts[26.0] = uni_marker_df.geometry[3.0]  # Add return path
chain_pts[27.0] = uni_marker_df.geometry[2.0]  # I haven't found marker 2 yet
chain_pts[28.0] = uni_marker_df.geometry[1.0]

chain_line_string = LineString(chain_pts)
chain_df = gp.GeoDataFrame(geometry=[chain_line_string])
chain_df.plot(ax=ax, linewidth=5, color="white")

# Calculate total length and add text
total_length = measure_line_string(chain_line_string)
ax.text(
    ScaleBarRenderer.move_point(geopy_pt(-33.9175, 151.265), 200, 0).longitude,
    geopy_pt(-33.9175, 151.265).latitude,
    f"The total length of the chain loop, from 1 back to 1, is {total_length:.0f} meters.",
    ha="left",
    fontsize=12,
    color=TEXT_COLOUR,
)
print(f"⛓️  Chain trail: {total_length:.0f}m total length")

# Add shortcut arrows using the refactored function
try:
    draw_shortcut_arrow(
        all_gdf,
        ax,
        from_marker_number=23,
        to_marker_number=5,
        text_colour=TEXT_COLOUR,
        arrow_colour=TEXT_COLOUR,
        text_size=9,
    )
    draw_shortcut_arrow(
        all_gdf,
        ax,
        from_marker_number=11,
        to_marker_number=16,
        text_colour=TEXT_COLOUR,
        arrow_colour=TEXT_COLOUR,
        text_size=9,
    )
    draw_shortcut_arrow(
        all_gdf,
        ax,
        from_marker_number=5,
        to_marker_number=14,
        text_colour=TEXT_COLOUR,
        arrow_colour=TEXT_COLOUR,
        text_size=9,
    )
    print("🧭 Added compass bearing shortcuts")
except Exception as e:
    print(f"⚠️  Could not add shortcut arrows: {e}")

# Add spear fishing boundary line
spear_fishing_boundary = gp.GeoSeries(
    LineString(SPEAR_FISHING_BOUNDARY_COORDS), crs=all_gdf.crs
)
spear_fishing_boundary.plot(ax=ax, color="red", linewidth=2, alpha=0.8)

# Create legend using the modular approach
legend_handles = marker_renderer.create_legend(ax)

# Calculate bounds and set limits
all_geometries = pd.concat([all_gdf.geometry, filtered_gdf.geometry])
buffer_radius = 0.0003
bounds = (
    Polygon(MultiPoint(all_geometries.values).envelope).buffer(buffer_radius).bounds
)

# Finalize plot using the map renderer
map_renderer.finalize_plot(bounds, "Gordon's Bay Trail Map")

# Save the map
plt.savefig("docs/marker_graph.png", transparent=True, dpi=300)
plt.savefig("docs/marker_graph_small.png", transparent=True, dpi=72)
plt.savefig("docs/marker_graph.svg", transparent=True)
plt.close()  # Keep performance optimized

print("🎉 Refactored map generation complete!")
print("💾 Saved as marker_graph_refactored.png and .svg")
print(
    f"📐 Map bounds: x=[{bounds[0]:.6f}, {bounds[2]:.6f}], y=[{bounds[1]:.6f}, {bounds[3]:.6f}]"
)

In [ ]:
# DIAGNOSTIC: Check available markers for shortcut arrows
print("🔍 SHORTCUT ARROW DIAGNOSTIC:")

# Check what markers we have
available_markers = sorted(uni_marker_df.index.tolist())
print(f"✓ Available markers: {available_markers}")

# Original map.ipynb shortcut arrows:
original_shortcuts = [
    (5, 14, "This should be 4 to 14, but I haven't found marker 4 yet"),
    (11, 16, "Standard shortcut"),
    (23, 5, "Return shortcut")
]

print(f"\n📋 Original shortcuts from map.ipynb:")
for from_marker, to_marker, note in original_shortcuts:
    from_exists = from_marker in available_markers
    to_exists = to_marker in available_markers
    status = "✅" if (from_exists and to_exists) else "❌"
    print(f"   {status} {from_marker} → {to_marker} (from: {from_exists}, to: {to_exists}) - {note}")

# Check the current refactored shortcuts
print(f"\n📋 Current refactored shortcuts:")
refactored_shortcuts = [(23, 5), (11, 16), (5, 14)]
for from_marker, to_marker in refactored_shortcuts:
    from_exists = from_marker in available_markers
    to_exists = to_marker in available_markers
    status = "✅" if (from_exists and to_exists) else "❌"
    print(f"   {status} {from_marker} → {to_marker} (from: {from_exists}, to: {to_exists})")

print(f"\n💡 Missing markers that prevent shortcut arrows:")
missing_markers = []
for from_marker, to_marker in refactored_shortcuts:
    if from_marker not in available_markers:
        missing_markers.append(from_marker)
    if to_marker not in available_markers:
        missing_markers.append(to_marker)

if missing_markers:
    print(f"   Missing: {sorted(set(missing_markers))}")
else:
    print("   None - all required markers exist!")
    print("   🤔 If arrows aren't showing, the issue might be:")
    print("      • Silent exceptions in the try/catch block")
    print("      • Arrows being drawn outside the map bounds") 
    print("      • Visual styling making them hard to see")
    print("      • Coordinate system issues")

In [ ]:
# DIAGNOSTIC: Check what's actually in the contour data
print("🔍 DIAGNOSTIC: Checking contour data integration")

# Check what contour_gdf contains
if 'contour_gdf' in locals():
    print(f"✓ contour_gdf exists with {len(contour_gdf)} records")
    print(f"✓ Depth range: {contour_gdf.depth.min():.1f}m to {contour_gdf.depth.max():.1f}m")
    
    # Check source breakdown
    if 'source' in contour_gdf.columns:
        source_counts = contour_gdf.source.value_counts()
        print(f"✓ Sources: {dict(source_counts)}")
    
    # Check for shore conditions specifically
    zero_depth = contour_gdf[contour_gdf.depth == 0]
    print(f"✓ Zero depth records (should be shore): {len(zero_depth)}")
    
    negative_half_depth = contour_gdf[contour_gdf.depth == -0.5] 
    print(f"✓ -0.5m depth records (should be protruding_bommie): {len(negative_half_depth)}")
    
    made_up_depths = contour_gdf[contour_gdf.depth.isin([-6, -9, -12])]
    print(f"✓ Made-up bottom depths (-6, -9, -12m): {len(made_up_depths)}")
else:
    print("❌ contour_gdf not found - contour data not properly prepared")

# Check if shore conditions were integrated
if 'click_gdf' in locals():
    print(f"✓ click_gdf loaded with {len(click_gdf)} shore condition records")
    conditions = click_gdf.bottom_condition.value_counts()
    print(f"✓ Shore conditions: {dict(conditions)}")
else:
    print("❌ click_gdf not found")

print("\n📋 COMPARISON with original map.ipynb requirements:")
print("   1. Shore conditions mapped to depths? (shore_rocks=0, beach=0, protruding_bommie=-0.5)")
print("   2. Shore data concatenated with underwater depth data?") 
print("   3. Contour generatirefon includes both datasets?")

In [ ]:
# FIX: Integrate shore conditions into contour data (like original map.ipynb)
print("🔧 FIXING: Integrating shore conditions into contour data...")

# Step 1: Prepare shore conditions with depths (exactly like original map.ipynb)
shore_conditions = list(SHORE_DEPTHS.keys())
shore_gdf = click_gdf[click_gdf.bottom_condition.isin(shore_conditions)].copy()
shore_gdf["depth"] = shore_gdf["bottom_condition"].map(SHORE_DEPTHS, na_action="ignore")

print(f"✓ Prepared shore conditions: {len(shore_gdf)} records")
print(f"✓ Shore depth mapping: {dict(shore_gdf.groupby('bottom_condition')['depth'].first())}")

# Step 2: Prepare underwater depth data (filter out shallow readings)
cropped_depths_gdf = contour_gdf[contour_gdf.depth < -1.5].copy()
print(f"✓ Underwater depth records: {len(cropped_depths_gdf)} records")

# Step 3: Concatenate shore and underwater data (the critical missing step!)
contour_gdf_fixed = pd.concat([cropped_depths_gdf, shore_gdf], ignore_index=True)

print(f"✅ FIXED contour data: {len(contour_gdf_fixed)} total records")
print(f"✓ New depth range: {contour_gdf_fixed.depth.min():.1f}m to {contour_gdf_fixed.depth.max():.1f}m")

# Verify the fix worked
zero_depth_fixed = contour_gdf_fixed[contour_gdf_fixed.depth == 0]
protruding_bommie_fixed = contour_gdf_fixed[contour_gdf_fixed.depth == -0.5]
print(f"✓ Shore conditions at 0m depth: {len(zero_depth_fixed)} records")
print(f"✓ Protruding bommie at -0.5m: {len(protruding_bommie_fixed)} records")

# Replace the old contour_gdf with the fixed one
contour_gdf = contour_gdf_fixed
print("🎉 Shore conditions now properly integrated for contour generation!")

# Export Depth Data

The final section exports depth data to CSV for further analysis or sharing.

In [ ]:
# Export depth data using the consolidated all_gdf dataset
try:
    # Use the consolidated dive data that includes all depth, GPS, and photo data
    export_data = all_gdf.copy()
    
    # Select key columns for export (using actual available columns)
    depth_records = export_data[export_data.depth.notnull() & (export_data.depth != 0)].copy()
    
    # Prepare export columns
    if hasattr(depth_records, 'geometry'):
        # Extract lat/lon from geometry
        depth_records['latitude'] = depth_records.geometry.apply(lambda x: x.y if x else None)
        depth_records['longitude'] = depth_records.geometry.apply(lambda x: x.x if x else None)
    
    # Select relevant columns for export
    export_columns = ['latitude', 'longitude', 'depth', 'description']
    available_columns = [col for col in export_columns if col in depth_records.columns]
    export_df = depth_records[available_columns].dropna()
    
    # Export to CSV
    output_file = "depth.csv"
    export_df.to_csv(output_file, index=True)  # Include timestamp index
    
    print(f"✅ Exported {len(export_df)} depth records to {output_file}")
    print(f"📊 Columns: {list(export_df.columns)}")
    if len(export_df) > 0:
        print(f"📅 Date range: {export_df.index.min()} to {export_df.index.max()}")
        print(f"🌊 Depth range: {export_df['depth'].max():.1f}m to {export_df['depth'].min():.1f}m")
    
except Exception as e:
    print(f"⚠️  Error exporting depth data: {e}")
    print("Available columns:", all_gdf.columns.tolist())
    print("Data shape:", all_gdf.shape)

# Interactive Folium Map

Create an interactive web map that can be saved as HTML for sharing. This section uses the same refactored data sources but presents them in an interactive format.

In [ ]:
# Create base map centered on Gordon's Bay using our consolidated data
# Extract coordinates from the all_gdf geometry column
coords_data = all_gdf[all_gdf.geometry.notnull()].copy()
coords_data['latitude'] = coords_data.geometry.apply(lambda x: x.y if x else None)
coords_data['longitude'] = coords_data.geometry.apply(lambda x: x.x if x else None)

center_lat = coords_data['latitude'].mean()
center_lon = coords_data['longitude'].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=17,
    tiles='CartoDB positron',  # Clean background for underwater features
    max_zoom=25,  # Allow extreme zoom levels
    min_zoom=1,   # Allow wide zoom out
    prefer_canvas=True  # Better performance for complex overlays
)

# Add satellite imagery overlay with extended zoom range
satellite = folium.raster_layers.WmsTileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    layers='World_Imagery',
    name='Satellite',
    overlay=True,
    control=True,
    transparent=False,
    opacity=0.8,
    max_zoom=25,  # Allow tiles to pixelate rather than disappear
    min_zoom=1,   # Show tiles at all zoom levels
    bounds=None,  # No bounds restriction
    show=True     # Show by default
)
satellite.add_to(m)

# Add an additional high-resolution satellite layer for better zoom behavior
# This uses Google's satellite tiles which often have better zoom behavior
try:
    google_satellite = folium.raster_layers.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr='Google Satellite',
        name='Google Satellite (High Res)',
        overlay=True,
        control=True,
        max_zoom=25,  # Allow extreme pixelation
        min_zoom=1
    )
    google_satellite.add_to(m)
    print("🛰️  Added Google Satellite layer for better zoom persistence")
except Exception as e:
    print(f"⚠️  Could not add Google Satellite layer: {e}")

# Add GPS tracks using our consolidated dive data - each dive track on its own layer
# Group by description to get individual dive tracks
for desc in dives_gdf.description.unique():
    track_data = dives_gdf[dives_gdf.description == desc]
    if not track_data.empty:
        # Create a feature group for this dive track
        track_layer = folium.FeatureGroup(name=f"🤿 GPS Track: {desc}")
        
        # Convert to list of [lat, lon] for folium
        track_coords = [[row.geometry.y, row.geometry.x] 
                       for _, row in track_data.iterrows() if row.geometry is not None]
        
        if track_coords:  # Only add if we have valid coordinates
            folium.PolyLine(
                locations=track_coords,
                color=config.TEXT_COLOUR,
                weight=2,
                opacity=0.8,
                popup=f"GPS Track: {desc}"
            ).add_to(track_layer)
            
        track_layer.add_to(m)

# Add bottom condition markers on their own layer
bottom_conditions_layer = folium.FeatureGroup(name="🪨 Bottom Conditions")
for _, marker in filtered_gdf.iterrows():
    # Get marker properties
    lat, lon = marker.geometry.centroid.y, marker.geometry.centroid.x
    condition = marker.get('bottom_condition', 'Unknown')
    
    # Use color from config
    marker_color = config.BOTTOM_CONDITION_COLORS.get(condition, 'gray')
    
    # Create popup text
    popup_text = f"""
    <b>Bottom Condition:</b> {condition}<br>
    <b>Coordinates:</b> {lat:.6f}, {lon:.6f}<br>
    <b>Marker ID:</b> {marker.name}
    """
    
    folium.CircleMarker(
        location=[lat, lon],
        radius=8,
        popup=folium.Popup(popup_text, max_width=200),
        color='white',
        fillColor=marker_color,
        fillOpacity=0.8,
        weight=2
    ).add_to(bottom_conditions_layer)

bottom_conditions_layer.add_to(m)

# Add spear fishing boundary on its own layer
boundaries_layer = folium.FeatureGroup(name="🎣 Spear Fishing Boundary")

folium.PolyLine(
    locations=SPEAR_FISHING_BOUNDARY_COORDS,
    color='red',
    weight=3,
    opacity=0.8,
    popup="Spear Fishing Boundary"
).add_to(boundaries_layer)

boundaries_layer.add_to(m)

# Add the chain trail on its own layer (the most important feature!)
chain_trail_layer = folium.FeatureGroup(name="⛓️ Chain Trail")
if 'chain_line_string' in locals() and chain_line_string is not None:
    # Extract coordinates from the LineString geometry
    chain_coords = [[point[1], point[0]] for point in chain_line_string.coords]  # [lat, lon] for folium
    folium.PolyLine(
        locations=chain_coords,
        color='orange',
        weight=5,
        opacity=0.9,
        popup="Gordon's Bay Chain Trail - The main navigation route"
    ).add_to(chain_trail_layer)
    print(f"⛓️  Added chain trail with {len(chain_coords)} points")
else:
    print("⚠️  Chain trail not available - run the matplotlib map cell first")

chain_trail_layer.add_to(m)

# Add numbered chain markers on their own layer
chain_markers_layer = folium.FeatureGroup(name="🎯 Chain Markers")
if 'uni_marker_df' in locals() and not uni_marker_df.empty:
    # TODO: don't show numbers or markers on the points that are just changes in direction of the chain. 
    # It's currently showing two markers for 6, for example. In the uni_marker_df if the marker_number 
    # column isn't a whole number, then it's a change in direction of the chain, and we don't want to 
    # show it as having a marker or a number, but we do want to show it controling the line (i.e. as a vertex).
    chain_marker_count = 0
    for marker_num, row in uni_marker_df.iterrows():
        if float(marker_num).is_integer():
            lat, lon = row.geometry.y, row.geometry.x
            
            # Add the circular marker
            folium.CircleMarker(
                location=[lat, lon],
                radius=12,
                popup=folium.Popup(f"<b>Chain Marker {marker_num}</b><br>Depth: {row.depth:.1f}m", max_width=200),
                color='orange',
                fillColor='yellow',
                fillOpacity=0.8,
                weight=3
            ).add_to(chain_markers_layer)
            
            # Add the number as a text label using DivIcon
            folium.Marker(
                location=[lat, lon],
                icon=folium.DivIcon(
                    html=f'<div style="font-family: Arial; font-weight: bold; font-size: 14px; color: black; text-align: center; text-shadow: 1px 1px 2px white;">{int(marker_num)}</div>',
                    icon_size=(20, 20),
                    icon_anchor=(10, 10)
                )
            ).add_to(chain_markers_layer)
            chain_marker_count += 1
    print(f"🎯 Added {chain_marker_count} numbered chain markers with visible numbers")

chain_markers_layer.add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save interactive map
m.save('clickmap_refactored.html')

print("🗺️  Interactive map saved as 'clickmap_refactored.html'")
print("📍 Features organized by layers:")
print("   🛰️  Satellite imagery (ArcGIS + Google)")
print("   🤿 Individual GPS dive tracks")
print("   🪨 Bottom condition markers")  
print("   🎣 Spear fishing boundary")
print("   ⛓️  Chain trail line")
print("   🎯 Chain markers with numbers")
print("💡 Use the layer control (top right) to toggle features on/off!")

# Display the map in the notebook
m

# Conclusion

This refactored notebook demonstrates the same dive mapping functionality as the original, but with improved:

## 🏗️ **Architecture**
- **Modular Design**: Separate classes for data loading, visualization, and processing
- **Configuration Management**: Centralized settings in `config.py`
- **Type Safety**: Proper type hints and error handling
- **Reusability**: Components can be easily reused in other projects

## 📊 **Data Processing**
- **Unified Data Loader**: `DiveDataProcessor` handles all data sources
- **Consistent Formatting**: Standardized coordinate systems and data structures
- **Error Handling**: Robust handling of missing or malformed data files

## 🎨 **Visualization**
- **Flexible Renderers**: Separate classes for different map elements
- **Consistent Styling**: Centralized color and styling configuration
- **Multiple Outputs**: Both static (PNG/SVG) and interactive (HTML) maps

## 🔍 **Original Comments Preserved**
All informational comments from the original notebook have been preserved, including:
- GPS precision explanations
- Data processing rationale
- Coordinate system details
- Bottom condition classifications

## 📈 **Benefits Achieved**
- **Maintainability**: Easier to modify individual components
- **Readability**: Clear separation of concerns and consistent code style
- **Extensibility**: New data sources or visualization types can be added easily
- **Testing**: Individual components can be unit tested
- **Documentation**: Better inline documentation and type hints

The refactored code produces identical results to the original while being significantly more maintainable and professional in structure.